In [1]:
sc

org.apache.spark.SparkContext@a97002dc

In [2]:
sc.version

2.1.0

# positions-geographiques-des-stations-du-reseau-ratp

In [3]:
// The code was removed by DSX for sharing.

+-------+--------------------+--------------------+--------------------+------------------+------------------+----------+-----------+
|stop_id|           stop_name|           stop_desc|               coord|          stop_lat|          stop_lon|code_INSEE|departement|
+-------+--------------------+--------------------+--------------------+------------------+------------------+----------+-----------+
|   2158|       Achères-Ville|Avenue de Conflan...|48.9700771763, 2....|48.970077176304514|2.0776181820083806|     78005|         78|
|   2159|              Alésia|Place Victor et H...|48.8280660197, 2....| 48.82806601968645| 2.326827420050836|     75114|         75|
|   2172|            Concorde|Tuileries (jardin...|48.8654893909, 2....|  48.8654893908901| 2.321411789213801|     75108|         75|
|   2174|          Convention|Vaugirard (337 ru...|48.8371369496, 2....| 48.83713694956265| 2.296396090155559|     75115|         75|
|   2178|Courcelle-sur-Yvette|Rue Fernand Leger...|48.70076301

# Validatations

In [4]:
// TEST Date cast
import java.text.SimpleDateFormat
val jourFormat = new SimpleDateFormat("yyyy-MM-dd")
print(jourFormat.parse("2016-01-05"))


Tue Jan 05 00:00:00 CST 2016

## "Moins de 5" using RDD

In [11]:
import java.sql.Date
val rdd_csv = sc.textFile("swift://sncf." + name + "/validations-sur-le-reseau-ferre-nombre-de-validations-par-jour-1er-semestre-2015.csv")
val headerAndRows = rdd_csv.map(line => line.split(";").map(_.trim))
val header = headerAndRows.first
// filter out header (eh. just check if the first val matches the first header name)
val data = headerAndRows.filter(_(0) != header(0))
// splits to map (header/value pairs)
val maps = data.map(splits => header.zip(splits).toMap)
// filter out the NB_VALD "Moins de 5"
val result = maps.filter(map => map("NB_VALD") != "Moins de 5")
// print result
result.take(1).foreach(println)

val columns=result.take(1).flatMap(a=>a.keys)
val resu=result.map{value=>
      val list=value.values.toList
      (list(0),list(1),list(2),Date.valueOf(list(3)),list(4),list(5).toDouble,list(6),list(7))
      }.toDF(columns:_*)

resu.show(5)
resu.printSchema()


1
Map(LIBELLE_ARRET -> SAINT-DENIS-PORTE DE PARIS, CODE_STIF_TRNS -> 100, CODE_STIF_ARRET -> 762, JOUR -> 2016-01-05, CODE_STIF_RES -> 110, NB_VALD -> 264, ID_REFA_LDA -> 72285, CATEGORIE_TITRE -> AMETHYSTE)
+--------------------+--------------+---------------+----------+-------------+-------+-----------+---------------+
|       LIBELLE_ARRET|CODE_STIF_TRNS|CODE_STIF_ARRET|      JOUR|CODE_STIF_RES|NB_VALD|ID_REFA_LDA|CATEGORIE_TITRE|
+--------------------+--------------+---------------+----------+-------------+-------+-----------+---------------+
|SAINT-DENIS-PORTE...|           100|            762|2016-01-05|          110|  264.0|      72285|      AMETHYSTE|
|       SAINT-GEORGES|           100|            766|2016-01-05|          110|  103.0|      71366|            TST|
|       SAINT-JACQUES|           100|            768|2016-01-05|          110|  415.0|      71041|            TST|
|        SAINT-LAZARE|           100|            769|2016-01-05|          110|15406.0|      71370|    

## "Moins de 5" using DataFrame

In [12]:
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.types._

val customSchema = StructType(Array(
    StructField("JOUR", DateType, true),
    StructField("CODE_STIF_TRNS", StringType, true),
    StructField("CODE_STIF_RES", StringType, true),
    StructField("CODE_STIF_ARRET", StringType, true),
    StructField("LIBELLE_ARRET", StringType, true),
    StructField("ID_REFA_LDA", StringType, true),
    StructField("CATEGORIE_TITRE", StringType, true),
    StructField("NB_VALD", IntegerType, true)))
//-- Important : DROPMALFORMED
val dfValidation = spark.
    read.format("org.apache.spark.sql.execution.datasources.csv.CSVFileFormat").
    option("header", "true").
    schema(customSchema).
    option("delimiter",";").
    option("nullValue", "null").
    option("mode", "DROPMALFORMED").
    load("swift://sncf." + name + "/validations-sur-le-reseau-ferre-nombre-de-validations-par-jour-1er-semestre-2015.csv")
dfValidation.show(5)
dfValidation.printSchema()

+----------+--------------+-------------+---------------+--------------------+-----------+---------------+-------+
|      JOUR|CODE_STIF_TRNS|CODE_STIF_RES|CODE_STIF_ARRET|       LIBELLE_ARRET|ID_REFA_LDA|CATEGORIE_TITRE|NB_VALD|
+----------+--------------+-------------+---------------+--------------------+-----------+---------------+-------+
|2016-01-05|           100|          110|            762|SAINT-DENIS-PORTE...|      72285|      AMETHYSTE|    264|
|2016-01-05|           100|          110|            766|       SAINT-GEORGES|      71366|            TST|    103|
|2016-01-05|           100|          110|            768|       SAINT-JACQUES|      71041|            TST|    415|
|2016-01-05|           100|          110|            769|        SAINT-LAZARE|      71370|      IMAGINE R|  15406|
|2016-01-05|           100|          110|            777|SAINT-PAUL (LE MA...|      71222|    AUTRE TITRE|     88|
+----------+--------------+-------------+---------------+--------------------+--

## 1. Total, min, max, écart type de validations sur l'ensemble du réseau

In [18]:
// Register the DataFrame as a global temporary view
dfValidation.createOrReplaceTempView("validation")
spark.sql("desc validation").show()
val exo1=spark.sql("select MAX(NB_VALD) as max ,MIN(NB_VALD) as min,SUM(NB_VALD) as sum ,STD(NB_VALD) as std from validation")
exo1.show()


+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|           JOUR|     date|   null|
| CODE_STIF_TRNS|   string|   null|
|  CODE_STIF_RES|   string|   null|
|CODE_STIF_ARRET|   string|   null|
|  LIBELLE_ARRET|   string|   null|
|    ID_REFA_LDA|   string|   null|
|CATEGORIE_TITRE|   string|   null|
|        NB_VALD|      int|   null|
+---------------+---------+-------+

+------+---+---------+-----------------+
|   max|min|      sum|              std|
+------+---+---------+-----------------+
|117496|  5|839349975|3340.326185063387|
+------+---+---------+-----------------+



In [14]:
// Register the DataFrame as a global temporary view
resu.createOrReplaceTempView("validation")
spark.sql("desc validation").show()


+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|  LIBELLE_ARRET|   string|   null|
| CODE_STIF_TRNS|   string|   null|
|CODE_STIF_ARRET|   string|   null|
|           JOUR|     date|   null|
|  CODE_STIF_RES|   string|   null|
|        NB_VALD|   double|   null|
|    ID_REFA_LDA|   string|   null|
|CATEGORIE_TITRE|   string|   null|
+---------------+---------+-------+



In [15]:
val exo1=spark.sql("select MAX(NB_VALD) as max ,MIN(NB_VALD) as min,SUM(NB_VALD) as sum ,STD(NB_VALD) as std from validation")
exo1.show()

+--------+---+------------+------------------+
|     max|min|         sum|               std|
+--------+---+------------+------------------+
|117496.0|5.0|8.39349975E8|3340.3261850633626|
+--------+---+------------+------------------+



## 2. Total, min, max, écart type de validations sur l'ensemble du réseau¶

In [19]:
val exo2=spark.sql("select CATEGORIE_TITRE, MAX(NB_VALD) as max ,MIN(NB_VALD) as min,SUM(NB_VALD) as sum ,STD(NB_VALD) as std from validation group by CATEGORIE_TITRE")
exo2.show()

+---------------+------+---+---------+------------------+
|CATEGORIE_TITRE|   max|min|      sum|               std|
+---------------+------+---+---------+------------------+
|         NAVIGO|117496|  5|566037981|6741.0084156052835|
|     NON DEFINI|    14|  5|      439|1.8580651587930026|
|      IMAGINE R| 19848|  5|137940120|1432.1276500195768|
|            FGT|  6262|  5| 36333147|424.46848008771155|
|    AUTRE TITRE|  6961|  5| 15030393|347.00778001998583|
|            TST| 13332|  5| 63756820| 804.7275485319755|
|      AMETHYSTE|  2632|  5| 20251075|208.15668474338432|
+---------------+------+---+---------+------------------+



## 3. Total, min, max, écart type de validations par station

In [20]:
val exo3=spark.sql("select LIBELLE_ARRET, MAX(NB_VALD) as max ,MIN(NB_VALD) as min,SUM(NB_VALD) as sum ,STD(NB_VALD) as std from validation group by LIBELLE_ARRET")
exo3.show()

+--------------------+-----+---+-------+------------------+
|       LIBELLE_ARRET|  max|min|    sum|               std|
+--------------------+-----+---+-------+------------------+
|                EGLY|  376|  5|  61544| 108.6159953329052|
|   CENSIER-DAUBENTON| 6381|  7|1401786|1828.7771316737378|
|      LAGNY-THORIGNY| 4791| 17| 764461| 1087.679400414759|
|           SAINT-CYR| 3460| 11| 617425| 890.6392464580025|
|               PASSY| 7840|  5|1472529|2051.0996447084526|
|     GRAVIGNY-BALIZY|  464|  5|  69129|127.58094482342993|
|PONT-NEUF (LA MON...| 3404| 13| 460452| 658.8047639668467|
|  PORTE DE CHARENTON| 4509| 18| 962686|1194.3798961405444|
|           GARIBALDI| 7181| 10|1435081|1789.4180059677938|
|             TAVERNY|  847|  5| 157469|218.68868034448818|
|              HERICY|   45|  5|   4829|10.644892604990504|
|                OSNY|  338|  5|  42528| 84.07423890518126|
|    MAIRIE DE CLICHY|15588| 31|2979463|4376.6448721747065|
|   PRE SAINT-GERVAIS|  720| 13| 154165|

## 4. Total, min, max, écart type de validations par mois

In [22]:
val exo4=spark.sql("select MONTH(JOUR), MAX(NB_VALD) as max ,MIN(NB_VALD) as min,SUM(NB_VALD) as sum ,STD(NB_VALD) as std from validation group by MONTH(JOUR) order by 1")
exo4.show()

+-----------+------+---+---------+------------------+
|month(JOUR)|   max|min|      sum|               std|
+-----------+------+---+---------+------------------+
|          1|117496|  5|140726652|  3328.18187446804|
|          2|109239|  5|138403148|3367.7334658128448|
|          3|108998|  5|145065114| 3322.785100693589|
|          4|110219|  5|137832683| 3284.150133365392|
|          5|110976|  5|137937966|3235.9367153657245|
|          6|110287|  5|139384412|3505.9581337188856|
+-----------+------+---+---------+------------------+



## 5. Jour de l'année/Titre de transport/Station ayant le plus/le moins de validation (au total et par titre de transport)

In [41]:
val exo5=spark.sql("select JOUR,CATEGORIE_TITRE, LIBELLE_ARRET,NB_VALD from ( select JOUR,CATEGORIE_TITRE, LIBELLE_ARRET,NB_VALD, dense_rank() OVER (PARTITION BY CATEGORIE_TITRE ORDER BY NB_VALD DESC) as rank FROM validation ) tmp WHERE rank = 1")
exo5.show()

+----------+---------------+--------------------+-------+
|      JOUR|CATEGORIE_TITRE|       LIBELLE_ARRET|NB_VALD|
+----------+---------------+--------------------+-------+
|2016-01-22|         NAVIGO|LA DEFENSE-GRANDE...| 117496|
|2016-01-27|     NON DEFINI|BIBLIOTHEQUE FRAN...|     14|
|2016-02-05|      IMAGINE R|        SAINT-LAZARE|  19848|
|2016-03-12|            FGT|        GARE DU NORD|   6262|
|2016-03-14|    AUTRE TITRE|        GARE DE LYON|   6961|
|2016-03-12|            TST|        GARE DU NORD|  13332|
|2016-05-03|      AMETHYSTE|          BELLEVILLE|   2632|
+----------+---------------+--------------------+-------+



## 6. Géolocalisation des dix stations de métro ayant le plus/le moins de validations par titre de transport

## 7. Vous pouvez apporter une visualisation ou des indicateurs complémentaires